# Automated ML

In [1]:
import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.webservice import Webservice
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model

print("SDK version:", azureml.core.VERSION)

SDK version: 1.51.0


In [21]:
print(ws.datasets.keys())


KeysView({'TitanicDataset': DatasetRegistration(id='773fa54c-b394-4604-85fe-a8fea1fef0a8', name='TitanicDataset', version=1, description='', tags={})})


## Dataset

### Overview
The dataset chosen for this project is the one from [Kaggle Titanic Challenge](https://www.kaggle.com/c/titanic). 
In the famous Titanic shipwreck, some passengers were more likely to survive than others. The dataset presents information about 871 passengers and a column that states if they have survived or not. The model that we will create using automated ML predicts which passengers survived the Titanic shipwreck.

In [9]:
ws = Workspace.from_config()

# Name of the experiment
experiment_name = 'capstone_project'
experiment=Experiment(ws, experiment_name)

# Get the data of Kaggle Titanic Dataset
key = "TitanicDataset"
print(ws.datasets.keys())
if key in ws.datasets.keys(): 
    print(ws.datasets.keys())
    found = True
    dataset = ws.datasets[key] 

else:
    # Create AML Dataset and register it into Workspace
    example_data = 'https://raw.githubusercontent.com/clasimoes/nd00333-capstone/master/titanic_data/full_capstone.csv'
    dataset = Dataset.Tabular.from_delimited_files(example_data)
    #Register Dataset in Workspace
    dataset = dataset.register(workspace=ws,
                               name=key, create_new_version=True)


df = dataset.to_pandas_dataframe()
df.describe()

KeysView({})


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Q,S,male
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.066409,0.523008,0.381594,32.204208,0.086420,0.725028,0.647587
std,257.353842,0.486592,0.836071,13.244532,1.102743,0.806057,49.693429,0.281141,0.446751,0.477990
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400,0.000000,0.000000,0.000000
50%,446.000000,0.000000,3.000000,26.000000,0.000000,0.000000,14.454200,0.000000,1.000000,1.000000
75%,668.500000,1.000000,3.000000,37.000000,1.000000,0.000000,31.000000,0.000000,1.000000,1.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000,1.000000,1.000000


In [55]:
#### Change the data type of label column from decimal to integer

In [28]:
# from azureml.core import Workspace, Dataset
df['Survived'] = df['Survived'].astype(int)
updated_dataset = Dataset.Tabular.register_pandas_dataframe(
    dataframe=df,
    target=(ws.get_default_datastore(), 'TitanicData2'),
    name='TitanicData2',
    show_progress=True
)

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to TitanicData2/e1c6cf67-0963-42d5-827f-2d0abb4cab99/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


### Create a compute cluster

We create here a compute cluster to run the experiment. In this cluster, we provise 2-10 machines with the "STANDARD_DS12_V2" configuration.

In [11]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cluster-capstone"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS3_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=10)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 2, timeout_in_minutes = 10)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded..................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## AutoML Configuration

The task here is to classify if each passenger survived the Titanic shipwreck or not, what led us to provide the column "Survived" as the classification target.
Because we have 10 nodes in our cluster, we are able to run up to 9 concurrent iterations in our experiment (1 node is meant to be used by the "parent" experiment). We have chosen accuracy as the primary metric to be optimized. Featurization is set to auto, leaving data to be automatically scaled and normalized by AutoML. 

In [29]:
# TODO: Put your automl settings here
automl_settings = {"max_concurrent_iterations": 4,
                   "primary_metric" : 'accuracy'
}
project_folder = './capstone-project'

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             experiment_timeout_minutes = 20,
                             training_data=updated_dataset,
                             label_column_name="Survived",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             **automl_settings)

In [30]:
# TODO: Submit your experiment
# RemoteRun is an object of type AutoMLRun
remote_run = experiment.submit(automl_config, show_output=False)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
capstone_project,AutoML_054d496a-23c1-4a94-aace-2daf4dca2f3a,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

`RunDetails` widget shows the run status per job.

In [31]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

Here get the best model from the automl experiments and display all the properties of the model.



In [33]:
best_run_automl, fitted_model_automl = remote_run.get_output()
model_name = best_run_automl.properties['model_name']

Package:azureml-automl-runtime, training version:1.52.0.post1, current version:1.51.0.post1
Package:azureml-core, training version:1.52.0, current version:1.51.0
Package:azureml-dataprep, training version:4.11.4, current version:4.10.8
Package:azureml-dataprep-rslex, training version:2.18.4, current version:2.17.12
Package:azureml-dataset-runtime, training version:1.52.0, current version:1.51.0
Package:azureml-defaults, training version:1.52.0, current version:1.51.0
Package:azureml-interpret, training version:1.52.0, current version:1.51.0
Package:azureml-mlflow, training version:1.52.0, current version:1.51.0
Package:azureml-pipeline-core, training version:1.52.0, current version:1.51.0
Package:azureml-responsibleai, training version:1.52.0, current version:1.51.0
Package:azureml-telemetry, training version:1.52.0, current version:1.51.0
Package:azureml-train-automl-client, training version:1.52.0, current version:1.51.0.post1
Package:azureml-train-automl-runtime, training version:1.

In [34]:
# run
print(best_run_automl)

Run(Experiment: capstone_project,
Id: AutoML_054d496a-23c1-4a94-aace-2daf4dca2f3a_36,
Type: azureml.scriptrun,
Status: Completed)


In [35]:
fitted_model_automl.steps

[('datatransformer',
  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, task='classification')),
 ('prefittedsoftvotingclassifier',
  PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('18', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.7, eta=0.1, gamma=0.1, max_depth=9, max_leaves=511, n_estimators=25, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0, reg_lambda=1.7708333333333335, subsample=0.9, tree_method='auto'))], verbose=False)), ('10', Pipeline(memory=None, steps=[('sparsenormalizer', Normalizer(copy=True, norm='l1')), ('l

In [36]:
from pprint import pprint

def print_model(model, prefix=""):
    for step in model.steps:
        print(prefix + step[0])
        if hasattr(step[1], 'estimators') and hasattr(step[1], 'weights'):
            pprint({'estimators': list(
                e[0] for e in step[1].estimators), 'weights': step[1].weights})
            print()
            for estimator in step[1].estimators:
                print_model(estimator[1], estimator[0] + ' - ')
        else:
            pprint(step[1].get_params())
            print()

print_model(fitted_model_automl)

datatransformer
{'enable_dnn': False,
 'enable_feature_sweeping': True,
 'feature_sweeping_config': {},
 'feature_sweeping_timeout': 86400,
 'featurization_config': None,
 'force_text_dnn': False,
 'is_cross_validation': True,
 'is_onnx_compatible': False,
 'observer': None,
 'task': 'classification',
 'working_dir': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/capstone/code/Users/odl_user_242438/capstone '
                'project/starter_file'}

prefittedsoftvotingclassifier
{'estimators': ['18', '10', '1', '25', '14', '5', '9', '22', '12', '19'],
 'weights': [0.15384615384615385,
             0.07692307692307693,
             0.15384615384615385,
             0.07692307692307693,
             0.07692307692307693,
             0.07692307692307693,
             0.15384615384615385,
             0.07692307692307693,
             0.07692307692307693,
             0.07692307692307693]}

18 - standardscalerwrapper
{'class_name': 'StandardScaler',
 'copy': True,
 'module_name': 'sklear

In [37]:
best_run_automl.get_file_names()

['accuracy_table',
 'automl_driver.py',
 'confusion_matrix',
 'explanation/c7e564bd/classes.interpret.json',
 'explanation/c7e564bd/eval_data_viz.interpret.json',
 'explanation/c7e564bd/expected_values.interpret.json',
 'explanation/c7e564bd/features.interpret.json',
 'explanation/c7e564bd/global_names/0.interpret.json',
 'explanation/c7e564bd/global_rank/0.interpret.json',
 'explanation/c7e564bd/global_values/0.interpret.json',
 'explanation/c7e564bd/local_importance_values.interpret.json',
 'explanation/c7e564bd/per_class_names/0.interpret.json',
 'explanation/c7e564bd/per_class_rank/0.interpret.json',
 'explanation/c7e564bd/per_class_values/0.interpret.json',
 'explanation/c7e564bd/rich_metadata.interpret.json',
 'explanation/c7e564bd/true_ys_viz.interpret.json',
 'explanation/c7e564bd/visualization_dict.interpret.json',
 'explanation/c7e564bd/ys_pred_proba_viz.interpret.json',
 'explanation/c7e564bd/ys_pred_viz.interpret.json',
 'explanation/d2b081c8/classes.interpret.json',
 'expl

In [38]:
best_run_automl.get_metrics()

{'average_precision_score_macro': 0.8579304877017229,
 'AUC_macro': 0.8670637485315383,
 'precision_score_macro': 0.8356828006982215,
 'accuracy': 0.8417478152309613,
 'recall_score_weighted': 0.8417478152309613,
 'weighted_accuracy': 0.8566344223606561,
 'AUC_micro': 0.8841428520217394,
 'recall_score_micro': 0.8417478152309613,
 'f1_score_micro': 0.8417478152309613,
 'f1_score_macro': 0.8275170773583665,
 'balanced_accuracy': 0.82411820965432,
 'precision_score_micro': 0.8417478152309613,
 'log_loss': 0.4216060032013066,
 'precision_score_weighted': 0.8433549655637211,
 'matthews_correlation': 0.6595608348963032,
 'recall_score_macro': 0.82411820965432,
 'average_precision_score_micro': 0.8648455342146549,
 'average_precision_score_weighted': 0.8654994145090266,
 'norm_macro_recall': 0.6482364193086402,
 'AUC_weighted': 0.8670637485315383,
 'f1_score_weighted': 0.8403485361469988,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_054d496a-23c1-4a94-aace-2daf4dca2f3a_36

In [39]:
best_run_automl.properties

{'runTemplate': 'automl_child',
 'pipeline_id': '__AutoML_Ensemble__',
 'pipeline_spec': '{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'capstone_project\',\'compute_target\':\'cluster-capstone\',\'subscription_id\':\'9a7511b8-150f-4a58-8528-3e7d50216c31\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_054d496a-23c1-4a94-aace-2daf4dca2f3a_36","experiment_name":"capstone_project","workspace_name":"quick-starts-ws-242438","subscription_id":"9a7511b8-150f-4a58-8528-3e7d50216c31","resource_group_name":"aml-quickstarts-242438"}}]}',
 'training_percent': '100',
 'predicted_cost': None,
 'iteration': '36',
 '_aml_system_scenario_identification': 'Remote.Child',
 '_azureml.Comput

In [40]:
best_run_automl.properties['run_algorithm']

'VotingEnsemble'

In [41]:
best_run_automl.properties['run_properties']

'classification_labels=numpy.array([0, 1]'

In [45]:
best_run_automl.properties['pipeline_spec']

'{"pipeline_id":"__AutoML_Ensemble__","objects":[{"module":"azureml.train.automl.ensemble","class_name":"Ensemble","spec_class":"sklearn","param_args":[],"param_kwargs":{"automl_settings":"{\'task_type\':\'classification\',\'primary_metric\':\'accuracy\',\'verbosity\':20,\'ensemble_iterations\':15,\'is_timeseries\':False,\'name\':\'capstone_project\',\'compute_target\':\'cluster-capstone\',\'subscription_id\':\'9a7511b8-150f-4a58-8528-3e7d50216c31\',\'region\':\'southcentralus\',\'spark_service\':None}","ensemble_run_id":"AutoML_054d496a-23c1-4a94-aace-2daf4dca2f3a_36","experiment_name":"capstone_project","workspace_name":"quick-starts-ws-242438","subscription_id":"9a7511b8-150f-4a58-8528-3e7d50216c31","resource_group_name":"aml-quickstarts-242438"}}]}'

In [46]:
#Save the best model - download pkl and register model in Azure
model_name = 'best_model'
model = best_run_automl.register_model(model_name, model_path='outputs/model.pkl')

In [48]:
best_run_automl.download_file('outputs/scoring_file_v_1_0_0.py', 'automl_output/score.py')
best_run_automl.download_file('outputs/model.pkl', 'automl_output/model.pkl')

In [49]:
print(model)

Model(workspace=Workspace.create(name='quick-starts-ws-242438', subscription_id='9a7511b8-150f-4a58-8528-3e7d50216c31', resource_group='aml-quickstarts-242438'), name=best_model, id=best_model:1, version=1, tags={}, properties={})


## Model Deployment

Create an inference config and deploy the model as a web service.

In [50]:
service_name = 'capstone-automl-service'
inference_config = InferenceConfig(entry_script='automl_output/score.py')
service = Model.deploy(ws, service_name, [model], inference_config, overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-10-08 01:35:32+00:00 Creating Container Registry if not exists..
2023-10-08 01:45:33+00:00 Generating deployment configuration.
2023-10-08 01:45:34+00:00 Submitting deployment to compute.
2023-10-08 01:45:38+00:00 Checking the status of deployment capstone-automl-service..
2023-10-08 01:47:30+00:00 Checking the status of inference endpoint capstone-automl-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [51]:
print(service.state)

Healthy


Send a request to the web service you deployed to test it.

In [52]:
import json

data = {"data":
        [
          {
            "PassengerId": 812,
            "Pclass": 2,
            "Name": "Berriman, Mr. William John",
            "Age": 23.0,
            "SibSp": 0,
            "Parch": 0, 
            "Ticket": "28425",
            "Fare": 13.0,
            "Cabin": "",
            "Q": 0,
            "S": 1,
            "male": 1
          },
          {
            "PassengerId": 813,
            "Pclass": 1,
            "Name": "Lesurer, Mr. Gustave J",
            "Age": 35.0,
            "SibSp": 0,
            "Parch": 0, 
            "Ticket": "PC 17755",
            "Fare": 512.3292,
            "Cabin": "B101",
            "Q": 0,
            "S": 0,
            "male": 1
          }
      ]
    }

# Convert to JSON string
input_data = json.dumps(data)

Print the logs of the web service and delete the service

In [53]:
output = service.run(input_data)

In [54]:
print(output)

{"result": [0, 1]}


Delete the service:

In [25]:
service.delete()

In [26]:
print(service.state)

Deleting
